Este es un ejemplo extraído del tutorial "Logistic Regression in 
Python - A Step-by-Step Guide" del blog "Pragmatic Machine 
Learning in Python" por Nick McCullum. 

En este ejemplo se utiliza una versión reducida del dataset Titanic
obtenida del blog antes mencionado. El dataset original se encuentra 
disponible en Kaggle y es muy utilizado como un dataset introductorio 
para problemas de regresión logistica. 

El dataset contiene características de los 
pasajeros que se encontraban a bordo del Titanic. Las mismas serán 
utilizadas para predecir si cada pasajero sobrevivió o no al hundimiento
del Titanic.

<hr>  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
%matplotlib widget

layout = widgets.Layout(align_items = 'center')

<ol>
    <li>Se utiliza la libreria <b>pandas</b> para importar el archivo .csv</li>
</ol>

In [ ]:
titanic_data = pd.read_csv('titanic_train.csv')

<ol start="2">
    <li>Con el método <b>data()</b> se obtienen las primeras 5 filas del 
        dataframe
    </li>
</ol>

In [ ]:
titanic_data.head()

<ol start="2">
    <li>Con el método <b>columns</b> se obtienen los nombres de todas 
        las columnas del dataframe
    </li>
</ol>

In [ ]:
titanic_data.columns

<ul>
    <li><b>PassengerId</b> número identificador de cada pasajero.</li>
    <li><b>Survived</b> número binario que indica si el pasajero sobrevivió o no al hundimiento del Titanic. El valor 1 corresponde a que la persona sobrevivió, mientras que el valor 0 corresponde a que no lo hizo.</li>
    <li><b>Pclass</b> número que corresponde a la clase del pasajero, puede tener un valor de 1, 2 ó 3.</li>
    <li><b>Name</b> nombre del pasajero.</li>
    <li><b>Sex</b> sexo del pasajero (male, female).</li>
    <li><b>Age</b> edad (en años) del pasajero.</li>
    <li><b>SibSp</b> número de hermanos o esposos a bordo del barco.</li>
    <li><b>Parch</b> número de padres o hijos a bordo del barco.</li>
    <li><b>Ticket</b> número de ticket del pasajero.</li>
    <li><b>Fare</b> precio que el pasajero pagó por su ticket.</li>
    <li><b>Cabin</b> número de la cabina del pasajero.</li>
    <li><b>Embarked</b> puerto en el que el pasajero embarcó (C = Cherbourg, Q = Queenstown, S = Southampton)</li>
</ul>


<ol start="3">
    <li> Utilizando la librería <b>seaborn</b> es posible observar los datos 
        faltantes en el dataset, los cuales se indican con líneas blancas.
    </li>
</ol>

In [ ]:
plt.subplots(1,1,figsize = (4,5),tight_layout = True)
sns.heatmap(titanic_data.isnull(), cbar=False);

Se observa que la variable Cabin posee en su mayoría datos faltantes, por lo que
esta variable no será utilizada en el modelo. Por otro lado, la variable Age 
también posee datos faltantes, pero en menor cantidad. Por lo que se 
reemplazan las edades faltantes por la edad promedio de los pasajeros de la misma 
clase.

In [ ]:
def impute_missing_age(columns):
    age = columns[0]
    passenger_class = columns[1]
    
    if pd.isnull(age):
        if(passenger_class == 1):
            return titanic_data[titanic_data['Pclass'] == 1]['Age'].mean()
        elif(passenger_class == 2):
            return titanic_data[titanic_data['Pclass'] == 2]['Age'].mean()
        elif(passenger_class == 3):
            return titanic_data[titanic_data['Pclass'] == 3]['Age'].mean()
        
    else:
        return age

In [ ]:
titanic_data['Age'] = titanic_data[['Age', 'Pclass']].apply(impute_missing_age, axis = 1)

In [ ]:
titanic_data.drop('Cabin', axis=1, inplace = True)
titanic_data.head()

<ol start="4">
    <li> Con el método <b>get_dummies()</b> es posible obtener variables de tipo dummy, 
        es decir, variables que asignan un valor a cada categoría dentro de una 
        variable no numérica. Por ejemplo: al aplicar este método sobre la columna 
        Sex, se crearán otras 2 columnas denominadas male y female. Cada una de estas 
        columnas sólo contendrá números binarios. Un valor de 0 en la columna female 
        indicaría que el pasajero es hombre, e implicaría un valor de 1 en la 
        columna male.
    </li>
</ol>

In [ ]:
sex_data = pd.get_dummies(titanic_data['Sex'], drop_first = True)
embarked_data = pd.get_dummies(titanic_data['Embarked'], drop_first = True)

titanic_data = pd.concat([titanic_data, sex_data, embarked_data], axis = 1)


<ol start="5">
    <li> Una vez agregadas las columnas de variables dummies, es posible eliminar 
        las columnas de las variables categóricas correspondientes. A su vez, 
        también se pueden eliminar otras variables que no son influyentes en 
        la predicción de si el pasajero sobrevivió o no (Nombre, Número de 
        pasajero y Número de ticket).
    </li>
</ol>

In [ ]:
titanic_data.drop(['Name', 'PassengerId', 'Ticket', 'Sex', 'Embarked'], axis = 1, inplace = True)

In [ ]:
titanic_data.head()

<ol start="6">
    <li> Se divide el dataset en los datos de entrada (x) y los valores de salida (y).
    </li>
</ol>

In [ ]:
y_data = titanic_data['Survived']
y_data

In [ ]:
x_data = titanic_data.drop('Survived', axis = 1)
x_data

<ol start="6">
    <li> Se utiliza la librería <strong>scikit-learn</strong>. Específicamente 
        la función <strong>train_test_split</strong> para dividir de manera 
        aleatoria un set de datos. Así se obtienen un set de datos de entrenamiento 
        y otro de prueba.
    </li>
</ol>


In [ ]:
from sklearn.model_selection import train_test_split
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.3)

<ol start="7">
    <li> Se utiliza la clase <strong>LogisticRegression</strong>, de 
        la librería <strong>scikit-learn</strong>, para crear el modelo.
        Y luego se utilizan los métodos <b>fit</b> y <b>predict</b> para 
        entrenar el modelo con los datos de entrenamiento, y para predecir
        las salidas correspondientes a los datos de prueba.
    </li>
</ol>


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter= 500, C = 1e6)
model.fit(x_training_data, y_training_data)
predictions = model.predict(x_test_data)

<ol start="8">
    <li>La librería <strong>scikit-learn</strong> posee distintos 
        módulos que permiten analizar el desempeño del modelo, como
        <b>classification_report</b> y <b>confusion_matrix</b>. A su
        vez, la clase LogicRegression posee un método que devuelve
        el valor medio de la exactitud dado un set de datos de prueba
        con sus salidas reales.
    </li>
</ol>


In [ ]:
from sklearn.metrics import classification_report
classification_report(y_test_data, predictions)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test_data, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix = cm)

fig, ax = plt.subplots(figsize = (4,5),tight_layout = True)
disp.plot(ax = ax)
disp.im_.colorbar.remove()

In [ ]:
model.score(x_test_data,y_test_data)